In [1]:
import optuna 
from xgboost import XGBClassifier
from optuna.integration import XGBoostPruningCallback 
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


train_df = pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/train_clean.csv")
test_df =  pd.read_csv("/home/mwanikii/Documents/data_science_projects/financial_inclusivity/input/test_clean.csv")

#X_train, y_train and X_test
y = train_df["bank_account"]
X = train_df.drop(["bank_account"], axis=1)
X_test = test_df.drop(["bank_account"], axis=1)
y_test = test_df["bank_account"]
 
def objective(trial, X, y):
    parameter_grid = {

        "n_estimators": trial.suggest_categorical("n_estimators", [10000]),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "max_leaves": trial.suggest_int("max_leaves", 20, 3000, step=20),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "max_bin": trial.suggest_int("max_bin", 200, 300),
        "lambda": trial.suggest_int("lambda", 0, 100, step=5),
        "alpha": trial.suggest_int("alpha", 0, 100, step=5),
        "gamma": trial.suggest_float("gamma", 0, 15),
    }
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=5)
    cv_scores = np.empty(5)


    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        model = XGBClassifier(objective="binary:logistic", **parameter_grid)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_test, y_test)],
            early_stopping_rounds=100,
            callbacks=[XGBoostPruningCallback(trial, "validation_0-logloss")])

        preds = model.predict(X_test)
        cv_scores[idx] = log_loss(y_test, preds)

    return np.mean(cv_scores)

study = optuna.create_study(direction="minimize", study_name="XGB Classifier")
func = lambda trial: objective(trial, X, y)
study.optimize(func, n_trials=20)



/home/mwanikii/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 1.1build1 is an invalid version and will not be supported in a future release
  warnings.warn(
/usr/lib/python3/dist-packages/pkg_resources/__init__.py:116: PkgResourcesDeprecationWarning: 0.1.43ubuntu1 is an invalid version and will not be supported in a future release
  warnings.warn(
[I 2022-08-21 16:13:57,185] A new study created in memory with name: XGB Classifier


[0]	validation_0-logloss:0.59777


/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[1]	validation_0-logloss:0.53077
[2]	validation_0-logloss:0.48169
[3]	validation_0-logloss:0.44588
[4]	validation_0-logloss:0.41894
[5]	validation_0-logloss:0.39750
[6]	validation_0-logloss:0.38048
[7]	validation_0-logloss:0.36742
[8]	validation_0-logloss:0.35734
[9]	validation_0-logloss:0.34912
[10]	validation_0-logloss:0.34250
[11]	validation_0-logloss:0.33715
[12]	validation_0-logloss:0.33148
[13]	validation_0-logloss:0.32744
[14]	validation_0-logloss:0.32398
[15]	validation_0-logloss:0.32106
[16]	validation_0-logloss:0.31822
[17]	validation_0-logloss:0.31589
[18]	validation_0-logloss:0.31250
[19]	validation_0-logloss:0.31039
[20]	validation_0-logloss:0.30896
[21]	validation_0-logloss:0.30720
[22]	validation_0-logloss:0.30591
[23]	validation_0-logloss:0.30437
[24]	validation_0-logloss:0.30275
[25]	validation_0-logloss:0.30149
[26]	validation_0-logloss:0.30076
[27]	validation_0-logloss:0.30016
[28]	validation_0-logloss:0.29937
[29]	validation_0-logloss:0.29891
[30]	validation_0-loglo

/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/optuna/trial/_trial.py:590: UserWarning: The reported value is ignore

...

[0]	validation_0-logloss:0.60822


/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(
[I 2022-08-21 16:17:09,210] Trial 18 pruned. Trial was pruned at iteration 0.
/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/home/mwanikii/.local/lib/python3.10/site-packages/xgboost/sklearn.py:793: UserWarning: `callbacks` in `fit` method is deprecated for be

[0]	validation_0-logloss:0.68271


[I 2022-08-21 16:17:09,313] Trial 19 pruned. Trial was pruned at iteration 0.


	Best value (rmse): 3.87322
	Best params:
		n_estimators: 10000
		learning_rate: 0.20172781199546924
		max_leaves: 2560
		max_depth: 8
		max_bin: 280
		lambda: 40
		alpha: 5
		gamma: 3.3910506390918242


In [2]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")
    


	Best value (rmse): 3.87322
	Best params:
		n_estimators: 10000
		learning_rate: 0.20172781199546924
		max_leaves: 2560
		max_depth: 8
		max_bin: 280
		lambda: 40
		alpha: 5
		gamma: 3.3910506390918242
